In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import torch

sys.path.append(os.path.abspath("../.."))

from configs.general import GeneralConfig
from configs.autoencoder import AEConfig
from nn_architectures.autoencoder import Autoencoder, load_autoencoder
import AstroChemNet.data_processing as dp
from AstroChemNet.inference import Inference

In [ ]:
df = pd.read_hdf("../../data/grav_collapse_clean.h5", key="val")

autoencoder = load_autoencoder(Autoencoder, GeneralConfig, AEConfig, inference=True)

processing = dp.Processing(GeneralConfig, AEConfig)

inference = Inference(GeneralConfig, processing, autoencoder)

In [ ]:
unique_models = df['Model'].unique()
random_models = np.random.choice(unique_models, 100, replace=False)
subset_df = df[df['Model'].isin(random_models)].groupby('Model').first().reset_index()

In [ ]:
results = []
for idx, row in subset_df.iterrows():
    abundances = row[GeneralConfig.species].values.astype(np.float32).reshape(1, -1)
    processing.abundances_scaling(abundances)
    latents = inference.encode(abundances).cpu().numpy().flatten()
    result = {
        'Model': row['Model'],
        'Time': row['Time'],
        'Density': row['Density'],
        'gasTemp': row['gasTemp'],
        'Av': row['Av'],
        'Radfield': row['Radfield'],
    }
    for i in range(len(latents)):
        result[f'latent_{i}'] = latents[i]
    results.append(result)

output_df = pd.DataFrame(results)

In [ ]:
output_df